<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/main/Zaj%C4%99cia_22_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=4ff852910ea43a609d79274e04f858d19450d7b94aec1b0527def2288e60e4b6
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
from backtesting import Backtest
from backtesting.test import EURUSD, GOOG
from backtesting import Strategy
#from backtesting import TrailingStrategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [ ]:
EURUSD.head()
EURUSD.tail()

,Open,High,Low,Close,Volume
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065
2018-02-07 15:00:00,1.23427,1.23444,1.22904,1.22904,6143


In [ ]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
class SmaCross_withSL(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 25

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        # open position only if it is not already open
        if crossover(self.sma1, self.sma2):
          # get previous bar close
          price = self.data.Close
          # long/short SL and TP
          long_sl = price * 0.99
          long_tp = price * 1.05
          if self.position.is_short:
            self.position.close()
            self.buy(size=0.5, sl=long_sl, tp=long_tp)
          elif self.position.is_long:
            pass
          else:
            self.buy(size=0.5, sl=long_sl, tp=long_tp)

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        # open position only if it is not already open
        elif crossover(self.sma2, self.sma1):
          price = self.data.Close
          short_sl = price * 1.01
          short_tp = price * 0.95
          if(self.position.is_long):
            self.position.close()
            # get previous bar close
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)
          elif self.position.is_short:
            pass
          else:
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)


In [ ]:
bt = Backtest(EURUSD, SmaCross,
              cash=10000,
              exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       99.26
Equity Final [$]                  10037.86323
Equity Peak [$]                   10115.18329
Return [%]                           0.378632
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                    0.380144
Volatility (Ann.) [%]                6.460604
Sharpe Ratio                          0.05884
Sortino Ratio                        0.080555
Calmar Ratio                         0.051217
Max. Drawdown [%]                   -7.422169
Avg. Drawdown [%]                   -1.265356
Max. Drawdown Duration      209 days 15:00:00
Avg. Drawdown Duration       22 days 12:00:00
# Trades                                  263
Win Rate [%]                        39.163498
Best Trade [%]                       2.354824
Worst Trade [%]                     -1.699392
Avg. Trade [%]                    

In [ ]:
bt = Backtest(EURUSD, SmaCross_withSL,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       99.24
Equity Final [$]                  8068.259523
Equity Peak [$]                  10002.744319
Return [%]                         -19.317405
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                  -19.386372
Volatility (Ann.) [%]                3.256267
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.339541
Avg. Drawdown [%]                   -6.514658
Max. Drawdown Duration      292 days 01:00:00
Avg. Drawdown Duration       97 days 14:00:00
# Trades                                  205
Win Rate [%]                        20.487805
Best Trade [%]                       2.286821
Worst Trade [%]                     -1.865903
Avg. Trade [%]                    

In [ ]:
pip install yfinance

In [ ]:
opt = bt.optimize(n1=[5,7,9,11,13,15], n2=[10,15, 20,25,30,35,40], maximize='Sharpe Ratio',
                  constraint=lambda p: p.n1 < p.n2)
print(opt)

print(opt._strategy)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       99.72
Equity Final [$]                  10577.36027
Equity Peak [$]                   11025.27214
Return [%]                           5.773603
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                    5.807172
Volatility (Ann.) [%]                7.373141
Sharpe Ratio                         0.787612
Sortino Ratio                        1.235103
Calmar Ratio                         1.091355
Max. Drawdown [%]                   -5.321066
Avg. Drawdown [%]                   -0.374736
Max. Drawdown Duration       84 days 09:00:00
Avg. Drawdown Duration        3 days 11:00:00
# Trades                                  904
Win Rate [%]                        45.022124
Best Trade [%]                        1.50861
Worst Trade [%]                     -1.752387
Avg. Trade [%]                    

In [ ]:
import datetime as dt
import yfinance as yf

data = yf.download("MSFT", period="1mo")
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-20,422.000000,425.959991,420.660004,425.230011,425.230011,17860100
2024-03-21,429.829987,430.820007,427.160004,429.369995,429.369995,21296200
2024-03-22,429.700012,429.859985,426.070007,428.739990,428.739990,17636500
2024-03-25,425.239990,427.410004,421.609985,422.859985,422.859985,18060500
2024-03-26,425.609985,425.989990,421.350006,421.649994,421.649994,16725600
